In [1]:
import joblib
import pandas as pd
import unicodedata
import re
import pickle
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, mean_squared_error

In [2]:
pipeline_path = "/Users/fabiancordenod/code/fqbq69/BIMpredict-/pipeline/randomforestpoutrespipeline.pkl"
pipeline = joblib.load(pipeline_path)

In [3]:
# Charger le CSV
csv_path = "/Users/fabiancordenod/code/fqbq69/BIMpredict-/datatest/test21020/poutres21020.csv"
df_test = pd.read_csv(csv_path, delimiter=';', header=1)

# Nettoyer les noms de colonnes comme à l'entraînement
def clean_col(col):
    col = ''.join(c for c in unicodedata.normalize('NFD', col) if unicodedata.category(c) != 'Mn')
    col = col.lower()
    col = re.sub(r"[ \-\(\)]", "_", col)
    col = re.sub(r"[^a-z0-9_]", "", col)
    col = re.sub(r"_+", "_", col)
    col = col.strip("_")
    return col

df_test.columns = [clean_col(c) for c in df_test.columns]

In [4]:
df_test

,id,011ec_lot,012ec_ouvrage,013ec_localisation,014ec_mode_constructif,nom,ai,as,hauteur_totale,hauteur,...,titre_omniclass,numero_omniclass,description_de_lassemblage,code_dassemblage,marque_de_type,protection_contre_lincendie,cout,element_ngf,type_dattachement_de_fin,type_dattachement_de_debut
0,1036386,NaN,NaN,NaN,NaN,Po,"-173,78","-173,48","0,300000000000001","0,300000000000011",...,Beams,23.25.30.11.14.14,NaN,NaN,NaN,NaN,0,"167,15",NaN,NaN
1,1036570,NaN,NaN,NaN,NaN,Po,"-173,78","-173,48","0,300000000000001","0,300000000000011",...,Beams,23.25.30.11.14.14,NaN,NaN,NaN,NaN,0,"167,15",NaN,NaN
2,1036588,NaN,NaN,NaN,NaN,Po,"-173,78","-173,48","0,300000000000001","0,300000000000011",...,Beams,23.25.30.11.14.14,NaN,NaN,NaN,NaN,0,"167,15",NaN,NaN
3,1036604,NaN,NaN,NaN,NaN,Po,"-173,78","-173,48","0,300000000000001","0,300000000000011",...,Beams,23.25.30.11.14.14,NaN,NaN,NaN,NaN,0,"167,15",NaN,NaN
4,1036666,NaN,NaN,NaN,NaN,Po,"-173,78","-173,48","0,300000000000001","0,300000000000011",...,Beams,23.25.30.11.14.14,NaN,NaN,NaN,NaN,0,"167,15",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,1388351,NaN,NaN,NaN,NaN,Po,"-173,88","-173,38","0,500000000000025","0,499999999999943",...,Beams,23.25.30.11.14.14,NaN,NaN,NaN,NaN,0,"167,15",NaN,NaN
108,1389870,NaN,NaN,NaN,NaN,Po,"-173,88","-173,38","0,500000000000025","0,499999999999943",...,Beams,23.25.30.11.14.14,NaN,NaN,NaN,NaN,0,"167,15",NaN,NaN
109,1390125,NaN,NaN,NaN,NaN,Po,"-173,88","-173,38","0,500000000000025","0,499999999999943",...,Beams,23.25.30.11.14.14,NaN,NaN,NaN,NaN,0,"167,15",NaN,NaN
110,1390491,NaN,NaN,NaN,NaN,Po,"-174,295","-172,965","1,33000000000001","1,33000000000001",...,Beams,23.25.30.11.14.14,NaN,NaN,NaN,NaN,0,"167,15",NaN,NaN


In [5]:
# Les targets à prédire (après nettoyage)
targets = [
    "011ec_lot",
    "012ec_ouvrage",
    "013ec_localisation",
    "014ec_mode_constructif"
]

# Colonnes explicatives attendues par le pipeline
features = pipeline.named_steps['preprocessor'].feature_names_in_

# S'assurer que toutes les colonnes sont présentes
for col in features:
    if col not in df_test.columns:
        df_test[col] = np.nan
X_test = df_test[features].copy()

In [6]:
y_pred = pipeline.predict(X_test)

In [7]:
# Les targets à prédire (après nettoyage)
targets = [
    "011ec_lot",
    "012ec_ouvrage",
    "013ec_localisation",
    "014ec_mode_constructif"
]

# Mettre les prédictions dans un DataFrame
df_pred = pd.DataFrame(y_pred, columns=targets)

# Afficher les premières lignes
print(df_pred.head(50))

   011ec_lot 012ec_ouvrage 013ec_localisation 014ec_mode_constructif
0         GO        POUTRE          INTERIEUR         COULE EN PLACE
1         GO        POUTRE          INTERIEUR         COULE EN PLACE
2         GO        POUTRE          INTERIEUR         COULE EN PLACE
3         GO        POUTRE          INTERIEUR         COULE EN PLACE
4         GO        POUTRE          INTERIEUR         COULE EN PLACE
5         GO        POUTRE          INTERIEUR         COULE EN PLACE
6         GO        POUTRE          INTERIEUR         COULE EN PLACE
7         GO        POUTRE          INTERIEUR         COULE EN PLACE
8         GO        POUTRE          INTERIEUR         COULE EN PLACE
9         GO        POUTRE          INTERIEUR         COULE EN PLACE
10        GO        POUTRE          INTERIEUR         COULE EN PLACE
11        GO        POUTRE          INTERIEUR         COULE EN PLACE
12        GO        POUTRE          INTERIEUR         COULE EN PLACE
13        GO        POUTRE        

In [8]:
df_pred.to_csv("resultats_predictionspoutres21020.csv", index=False)